In [28]:
# Model name

# model_name = 'bert-large-uncased-whole-word-masking-finetuned-squad'
#  1.5 sek
#  Engelsk
#  1/ 2 poeng
#  Norsk
#  2/ 2 poeng


# model_name = 'ltg/norbert3-large'
#  2.3 sek
#  Engelsk
#  0?/ 2 poeng
#  Norsk
#  0?/ 2 poeng

# model_name = 'microsoft/DialoGPT-small'
#  0 sek
#  Engelsk
#  0/ 2 poeng
#  Norsk
#  0/ 2 poeng


model_name = ''
#  0 sek
#  Engelsk
#  0/ 2 poeng
#  Norsk
#  0/ 2 poeng




# model_name = 'lmsys/vicuna-13b-v1.5-16k'
#   sek
#  Engelsk
#  0?/ 2 poeng
#  Norsk
#  0?/ 2 poeng

# model_name = 'pankajmathur/orca_mini_v3_7b'
#   sek
#  Engelsk
#  0?/ 2 poeng
#  Norsk
#  0?/ 2 poeng



In [26]:
from transformers import AlbertTokenizer, AlbertForQuestionAnswering, AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForQuestionAnswering, AutoModelForCausalLM
import torch

model_name = 'bert-large-uncased-whole-word-masking-finetuned-squad'
model = AutoModelForQuestionAnswering.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name,  trust_remote_code=True)


In [29]:
def get_answer(text, question):
    # Tokenize the input text and question
    inputs = tokenizer(question, text, return_tensors='pt', max_length=512, truncation=True)
    outputs = model(**inputs)
    # Get the start and end scores for the answer
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
    # answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]))
    return answer





In [30]:
# Engelsk
context = """
Stig is the boss at Aboveit. He is a very capable boss, and sometimes he laughs at jokes.
"""
question = "Hello, who is the boss at Aboveit?"
answer = get_answer(context, question)
print(f"SHOULD BE 'Stig':        {answer}")

context = """
Stig is not the boss at Aboveit. He is a very capable boss, and sometimes he laughs at jokes.
"""
question = "Hello, who is the boss at Aboveit?"
answer = get_answer(context, question)
print(f"SHOULD NOT BE 'Stig':    {answer}")


context = """
Stig er sjef i Aboveit. Han er en veldig dyktig sjef, og av og til ler han av vitser.
"""
question = "Hei, hvem er sjefen i Aboveit?"
answer = get_answer(context, question)
print(f"Bør være 'Stig':         {answer}")

context = """
Stig er ikke sjef i Aboveit. Han er en veldig dyktig sjef, og av og til ler han av vitser.
"""
# bert-large-cased-whole-word-masking-finetuned-squad
answer = get_answer(context, question)
print(f"Bør ikke være 'Stig':    {answer}")





SHOULD BE 'Stig':        [CLS] hello, who is the boss at aboveit? [SEP] stig is the boss at aboveit. he is a very capable boss, and sometimes he laughs at jokes. [SEP]
SHOULD NOT BE 'Stig':    [CLS] hello, who is the boss at aboveit? [SEP] stig is not the boss at aboveit. he is a very capable boss, and sometimes he laughs at jokes. [SEP]
Bør være 'Stig':         [CLS] hei, hvem er sjefen i aboveit? [SEP] stig er sjef i aboveit. han er en veldig dyktig sjef, og av og til ler han av vitser. [SEP]
Bør ikke være 'Stig':    [CLS] hei, hvem er sjefen i aboveit? [SEP] stig er ikke sjef i aboveit. han er en veldig dyktig sjef, og av og til ler han av vitser. [SEP]


In [21]:
model_name = 'bert-large-uncased-whole-word-masking-finetuned-squad'
model = AutoModelForQuestionAnswering.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name,  trust_remote_code=True)


# AutoModelForCausalLM, som er designet for årsakssammenheng i språkmodellering (causal language modeling) og genererer tekst basert på tidligere kontekst. Denne modelltypen gir ikke start_logits og end_logits, som er nødvendig for spørsmål-svar-oppgaver.

# For spørsmål-svar-oppgaver, bør du bruke en modell som er spesifikt finjustert for dette formålet, som AutoModelForQuestionAnswering. Her er hvordan du kan endre koden for å bruke riktig modelltype:



# Angi stien hvor du vil lagre modellen og tokenizeren
save_directory = model_name

# Lagre modellen
model.save_pretrained(save_directory)

# Lagre tokenizeren
tokenizer.save_pretrained(save_directory)


model = AutoModelForCausalLM.from_pretrained(save_directory)
tokenizer = AutoTokenizer.from_pretrained(save_directory)


If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad and are newly initialized: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
